In [1]:
import pandas as pd, numpy as np
import pickle
import json
from tqdm import tqdm
from pandas import DataFrame
from collections import defaultdict

In [2]:
def read_pkl(path):
    with open(path, "rb") as f:
        t = pickle.load(f)
    return t

In [3]:
import collections
def reverse_dict(d):
    # {1:[a,b,c], 2:[a,f,g],...}
    re_d = collections.defaultdict(list)
    for k, v_list in d.items():
        for v in v_list:
            re_d[v].append(k)
    return dict(re_d)

In [4]:
def jsonKeys2int(x):
    if isinstance(x, dict):
        return {int(k):v for k,v in x.items()}
    return x

In [ ]:
state = 'meta_training'

support_u_movies = json.load(open(state+'/support_u_movies.json', 'r'), object_hook=jsonKeys2int)
query_u_movies= json.load(open(state+'/query_u_movies.json', 'r'), object_hook=jsonKeys2int)
support_u_movies_y = json.load(open(state+'/support_u_movies_y.json', 'r'), object_hook=jsonKeys2int)
query_u_movies_y = json.load(open(state+'/query_u_movies_y.json', 'r'), object_hook=jsonKeys2int)
if support_u_movies.keys() == query_u_movies.keys():
    u_id_list = support_u_movies.keys()
print(len(u_id_list))   #print the number of the users in meta-training

train_u_movies = {}
train_u_movies_y = {}
if support_u_movies.keys() == query_u_movies.keys():
    u_id_list = support_u_movies.keys()
print(len(u_id_list))
for idx, u_id in tqdm(enumerate(u_id_list)):
    train_u_movies[int(u_id)] = []
    train_u_movies_y[int(u_id)] = []
    train_u_movies[int(u_id)] += support_u_movies[u_id]+query_u_movies[u_id]
    train_u_movies_y[int(u_id)] += support_u_movies_y[u_id]+query_u_movies_y[u_id]
print(len(train_u_movies)) 

In [ ]:
with open('meta_training/train_u_movies.json', 'w') as json_file:
    json.dump(train_u_movies, json_file)
    
with open('meta_training/train_u_movies_y.json', 'w') as json_file:
    json.dump(train_u_movies_y, json_file)

In [ ]:
# get mp data 
print(state)

u_m_r_m_u = {}

#support_u_movies--->user: [movie1, movie2, movie3,.....]
#support_m_users--->movie: [user1, user2, user3,.....]

support_m_users = reverse_dict(support_u_movies)

for u in tqdm(u_id_list):
    
    u_movies = support_u_movies[u]
    u_ratings = support_u_movies_y[u]
    u_movie_rating_dict = dict(zip(u_movies, u_ratings))
    
    u_m_r_m_u[u] = set([])
    for m in support_u_movies[u]:  
        temp_rating = u_movie_rating_dict[m]
        for _u in support_m_users[m]:
            _u_movies = support_u_movies[_u]
            _u_ratings = support_u_movies_y[_u]
            _u_movie_rating_dict = dict(zip(_u_movies, _u_ratings))
            if _u_movie_rating_dict[m] == temp_rating:
                u_m_r_m_u[u].add(_u)
    u_m_r_m_u[u].discard(u)

In [ ]:
def Hamming_distance(array_A, array_B):

    smstr = np.nonzero(array_A - array_B)
    sm= np.shape(smstr[0])[0] 
    
    return sm

In [ ]:
all_user_property_array = read_pkl('user_property_array.pkl')

alpha = 0.5
topK = 10
user_feature_num = 4

u_similar_u = {}

for u in tqdm(list(u_m_r_m_u.keys())):
    
    u_movies = support_u_movies[u]
    u_ratings = support_u_movies_y[u]
    u_movie_rating_dict = dict(zip(u_movies, u_ratings))
        
    if len(u_m_r_m_u[u]) < topK:
        u_similar_u[u] = u_m_r_m_u[u]
        continue
    
    _u_list = []
    _u_error_list = []
    u_property_array = all_user_property_array[all_user_property_array['user_id']==u]['property_array'].iloc[0]
    
    for _u in u_m_r_m_u[u]:      
            
        _u_list.append(_u)
        
        _u_movies = support_u_movies[_u]
        _u_ratings = support_u_movies_y[_u]
        _u_movie_rating_dict = dict(zip(_u_movies, _u_ratings))
        
        common_movies = list(set(u_movies).intersection(_u_movies))
            
        rating_delta = []
        for i in common_movies:
            rating_delta.append(abs(u_movie_rating_dict[i]-_u_movie_rating_dict[i]))
        temp_rating_delta = np.mean(rating_delta)
        
        _u_property_array = all_user_property_array[all_user_property_array['user_id']==_u]['property_array'].iloc[0]
        user_content_hd = Hamming_distance(u_property_array, _u_property_array) / user_feature_num 
        sum_error = alpha * user_content_hd + (1 - alpha) * temp_rating_delta 
        
        _u_error_list.append(sum_error)
        
    _u_error_dict = dict(zip(_u_list, _u_error_list))
    _u_sorted_list1 = sorted(_u_error_dict.items(), key=lambda x : x[1])
    
    _u_sorted_list = list(map((lambda x : x[0]), _u_sorted_list1))
    _u_sorted_list_y = list(map((lambda x : x[1]), _u_sorted_list1))
    _u_sample = _u_sorted_list[:topK]
    _u_sample_y = _u_sorted_list_y[:topK]
    u_similar_u[u] = _u_sample

u_similar_u_dict = defaultdict(list)

for u in tqdm(u_similar_u.keys(), total=len(u_similar_u)):
    u_similar_u_dict[u] = list(u_similar_u[u])
    
with open('meta_training/support_u_similar_u_alpha{}.json'.format(alpha), 'w') as json_file:
    json.dump(u_similar_u_dict, json_file)

In [ ]:
state = 'user_and_item_cold_testing'

support_u_movies = json.load(open(state+'/support_u_movies.json','r'), object_hook=jsonKeys2int)
query_u_movies= json.load(open(state+'/query_u_movies.json','r'), object_hook=jsonKeys2int)
support_u_movies_y = json.load(open(state+'/support_u_movies_y.json','r'), object_hook=jsonKeys2int)
query_u_movies_y = json.load(open(state+'/query_u_movies_y.json','r'), object_hook=jsonKeys2int)
if support_u_movies.keys() == query_u_movies.keys():
    u_id_list = support_u_movies.keys()
print(len(u_id_list))

cur_train_u_movies = train_u_movies.copy()
cur_train_u_movies_y = train_u_movies_y.copy()

if support_u_movies.keys() == query_u_movies.keys():
    u_id_list = support_u_movies.keys()
print(len(u_id_list))

for idx, u_id in tqdm(enumerate(u_id_list)):
    if u_id not in cur_train_u_movies:
        cur_train_u_movies[u_id] = []
        cur_train_u_movies_y[u_id] = []
    cur_train_u_movies[u_id] += support_u_movies[u_id]
    cur_train_u_movies_y[u_id] += support_u_movies_y[u_id]

print(len(cur_train_u_movies),  len(train_u_movies))

In [ ]:
# get mp data 
print(state)

u_m_r_m_u = {}

#cur_train_u_movies--->user: [movie1, movie2, movie3,.....]
#cur_train_m_users--->movie: [user1, user2, user3,.....]

cur_train_m_users = reverse_dict(cur_train_u_movies)

for u in tqdm(u_id_list):
    
    u_movies = cur_train_u_movies[u]
    u_ratings = cur_train_u_movies_y[u]
    u_movie_rating_dict = dict(zip(u_movies, u_ratings))
    
    u_m_r_m_u[u] = set([])

    for m in cur_train_u_movies[u]:  
        
        temp_rating = u_movie_rating_dict[m]
        
        for _u in cur_train_m_users[m]:
            _u_movies = cur_train_u_movies[_u]
            _u_ratings = cur_train_u_movies_y[_u]
            _u_movie_rating_dict = dict(zip(_u_movies, _u_ratings))
            if _u_movie_rating_dict[m] == temp_rating:
                 u_m_r_m_u[u].add(_u)
                    
    u_m_r_m_u[u].discard(u)

In [ ]:
alpha = 0.5
topK = 10
user_feature_num = 4

u_similar_u = {}

for u in tqdm(list(u_m_r_m_u.keys())):
    
    u_movies = cur_train_u_movies[u]
    u_ratings = cur_train_u_movies_y[u]
    u_movie_rating_dict = dict(zip(u_movies, u_ratings))

    if len(u_m_r_m_u[u]) < topK:
        u_similar_u[u] = u_m_r_m_u[u]
        continue
    
    _u_list = []
    _u_error_list = []
    u_property_array = all_user_property_array[all_user_property_array['user_id']==u]['property_array'].iloc[0]
    
    for _u in u_m_r_m_u[u]:

        _u_list.append(_u)
        
        _u_movies = cur_train_u_movies[_u]
        _u_ratings = cur_train_u_movies_y[_u]
        _u_movie_rating_dict = dict(zip(_u_movies, _u_ratings))
        
        common_movies = list(set(u_movies).intersection(_u_movies))
        rating_delta = []
        for i in common_movies:
            rating_delta.append(abs(u_movie_rating_dict[i]-_u_movie_rating_dict[i]))
        temp_rating_delta = np.mean(rating_delta)
        
        _u_property_array = all_user_property_array[all_user_property_array['user_id']==_u]['property_array'].iloc[0]
        user_content_hd = Hamming_distance(u_property_array, _u_property_array) / user_feature_num
        
        sum_error = alpha * user_content_hd + (1 - alpha) * temp_rating_delta 
        _u_error_list.append(sum_error)
        
    _u_error_dict = dict(zip(_u_list, _u_error_list))
    _u_sorted_list = sorted(_u_error_dict.items(), key=lambda x : x[1])
    
    #print(_u_sorted_list1[:10])
    
    _u_sorted_list = list(map((lambda x : x[0]), _u_sorted_list))
    _u_sample = _u_sorted_list[:topK]
    u_similar_u[u] = _u_sample
    
u_similar_u_dict = defaultdict(list)

for i in tqdm(u_similar_u.keys(), total=len(u_similar_u)):
    u_similar_u_dict[i] = list(u_similar_u[i])
    
with open('{}/u_similar_u_alpha{}.json'.format(state, alpha), 'w') as json_file:
    json.dump(u_similar_u_dict, json_file)